In [1]:
import pandas as pd

In [2]:
posts = pd.read_parquet('items.parquet')
posts.head()

,authorId,content,itemId,title
0,22062724202903043,Британский кот — олицетворение невозмутимости....,1155255918526217609,"Как воспитать британского кота добрым, ласковы..."
1,5021973993129217102,Комнатные растения можно встретить в квартирах...,3466678402942852201,Вызывают судороги и смерть: пять опасных комна...
2,5843309594367696221,КОНТЕЙНЕР И БЫТОВКА Бывший в употреблении морс...,9802178366207388656,Из чего построить дачный дом? Семь доступных в...
3,4785891415385376217,Советский разведчик Аркадий Корнев с гранатой ...,5959787242217250844,Как осажденные советские солдаты в Афганистане...
4,6405657875686126019,Преимущества и недостатки оснастки и техники...,14212405135790401725,Преимущества и недостатки оснастки и техники д...


In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('cointegrated/rubert-tiny2')
# model = SentenceTransformer('ai-forever/sbert_large_nlu_ru')
embeddings = model.encode(["Привет", "Здравствуйте"])

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:
import torch
from torch.utils.data import DataLoader, Dataset

In [ ]:
class textDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        df['text'] = df['content'].fillna('') + ' ' + df['title'].fillna('')
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        return {
            'text': self.df.iloc[index]['text']
        }

In [42]:
text_set = textDataset(posts)
text_loader = DataLoader(text_set, shuffle=False, batch_size=512)

In [35]:
from tqdm import tqdm

In [ ]:
all_embs = []

for batch in tqdm(text_loader):
    texts = batch['text']
    with torch.no_grad():
        emb = model.encode(texts, convert_to_numpy=True)
    all_embs.append(emb)

import numpy as np
embeddings = np.vstack(all_embs)

  0%|          | 1/488 [00:48<6:32:16, 48.33s/it]


KeyboardInterrupt: 

круто, но бессмысленно

In [ ]:
embeddings = model.encode(
    posts['content'].fillna('').str[:3000] + ' ' + posts['title'].str[:200].fillna(''),
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    device='cuda'
)

Batches:   0%|          | 0/15591 [00:00<?, ?it/s]

In [5]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [11]:
emb_df = pd.DataFrame(embeddings, columns=[f'emb_{i}' for i in range(embeddings.shape[1])])

emb_df['itemId'] = posts['itemId'].values

In [ ]:
emb_df.to_parquet('embeds.parquet', index=False)